In [1]:
pip install langid pandas

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.9/1.9 MB 817.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941173 sha256=630104ded5583b3f9b33db0db50ea247d46437fb7f603e397824dc4ee097d696
  Stored in directory: c:\users\shruti tripathi\appdata\local\pip\cache\wheels\93\95\a9\c292c9dd8cadb8f2359f1670ff198a40d47167b0be3236e1c8
Successfully built langid
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import numpy as np 
import pandas as pd
import langid

In [2]:
df = pd.read_csv('spotify_songdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df.shape

(57650, 4)

In [6]:
df = df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [7]:
df.head(10)

,artist,song,text
0,Bette Midler,Heart Over Head,I had a will of iron \r\nI had a heart of ste...
1,Andy Williams,The Most Wonderful Time Of The Year,It's the most wonderful time of the year \r\n...
2,Incubus,Pantomime,"In my fantasy, I'm a pantomime \r\nI'll just ..."
3,Within Temptation,Mother Earth,Birds and butterflies \r\nRivers and mountain...
4,Yolanda Adams,The Only Way,Everywhere that I go \r\nI must tell everyone...
5,Rod Stewart,Good Morning Little Schoolgirl,Good morning little schoolgirl \r\nGood morni...
6,Youngbloodz,Tequila,\r\n[Intro] \r\nHey what's happenin' baby ...
7,Lionel Richie,Sweet Love - Commodores,Show me a river that's so deep \r\nShow me a ...
8,Dave Matthews Band,Long Black Veil,Ten years ago on a cold dark night \r\nSomeon...
9,Alabama,The Refrain Of John Dillon James,"\r\nOh, John Dillon James was my good time c..."


In [8]:
df.shape

(5000, 3)

In [9]:
df.isnull().sum()

artist    0
song      0
text      0
dtype: int64

In [10]:
#Cleaning

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)
df['text'][0]

'i had a will of iron  \r i had a heart of steel  \r i always call the shots  \r but, honey, you shoot to kill  \r i\'ve spent a hundred other nights like this  \r a little laughter and a stranger\'s kiss  \r   \r but the next thing that i knew  \r i was heart over head  \r over heart over head over you  \r it\'s too late to stop it now  \r i\'m heart over head  \r over heart over head over you  \r   \r i never once imagined  \r a night like this before  \r that love would last for hours  \r and still i begged for more  \r if i sound a little crazy  \r well, forgive me, please  \r my head is swimmin\'  \r and i\'m on my knees  \r   \r cryin\', "love me, love me, do!"  \r because i\'m heart over head  \r over heart over head over you  \r too late to stop it now  \r i\'m heart over head  \r over heart over head over you  \r   \r can\'t seem to slow down  \r can\'t even touch the ground  \r the harder i try  \r the higher i fly  \r   \r i\'m fallin\' heart over head  \r over heart over he

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
y=[]

def tokenization(text):
    tokens = nltk.word_tokenize(text)
    y=[ps.stem(word) for word in tokens]
    
    return " ".join(y)
    

In [14]:
df['text'] = df['text'].apply(lambda x : tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfid = TfidfVectorizer(stop_words = 'english')
matrix = tfid.fit_transform(df['text'])

In [17]:
matrix.shape

(5000, 17459)

In [18]:
similarity_mat = cosine_similarity(matrix)

In [19]:
similarity_mat[0]

array([1.        , 0.03815419, 0.00525942, ..., 0.02360567, 0.04496232,
       0.01293936])

In [26]:
def recommend(song_name):
    index = df[df['song']==song_name].index[0]
    distances = similarity_mat[index]
    song_list = sorted(list(enumerate(distances)),reverse=True,key= lambda x :x[1])[1:11]
    
    for i in song_list:
        print(df.iloc[i[0]].song)

In [27]:

recommend('Heart Over Head')

Head And Heart
A Head Full Of Dreams
Head Games
Heart Of Hearts
Heart Of Hearts
Let Me In Your Heart Again
Key 2 Your Heart
Left 2 3 4
I Wish My Heart Could Talk
Head Held High


In [22]:
df.to_dict()

{'artist': {50417: 'Reba Mcentire',
  21537: 'Willie Nelson',
  21161: 'Waylon Jennings',
  55176: 'Vince Gill',
  54611: 'Uriah Heep',
  27519: 'Chris Brown',
  37286: 'Israel',
  20721: 'Venom',
  8977: 'The Jam',
  28638: 'Dan Fogelberg',
  49386: 'Queen Adreena',
  56192: 'Whitesnake',
  4814: 'Ellie Goulding',
  39912: 'Kenny Chesney',
  5540: 'Evanescence',
  8582: 'Insane Clown Posse',
  38143: 'John McDermott',
  55667: 'Waylon Jennings',
  43465: 'Megadeth',
  1257: 'Beautiful South',
  57462: 'Young Buck',
  34254: 'GMB',
  45290: 'Neil Diamond',
  50214: 'Ray Boltz',
  19713: 'Tragically Hip',
  25151: 'Bette Midler',
  32483: 'Fall Out Boy',
  6226: 'Flo-Rida',
  48888: 'Poison',
  23368: 'Alice Cooper',
  9479: 'John Martyn',
  23669: 'Allman Brothers Band',
  15674: 'Pearl Jam',
  54878: 'Vanilla Ice',
  11101: 'Lady Gaga',
  28487: 'Culture Club',
  21157: 'Waylon Jennings',
  4029: 'Def Leppard',
  39519: 'Katy Perry',
  31270: 'Elvis Costello',
  3197: 'Cliff Richard',

In [40]:
import pickle

In [41]:
pickle.dump(df.to_dict(),open('song_dict2.pkl','wb'))

In [42]:
pickle.dump(similarity_mat,open('similarity2.pkl','wb'))